In [1]:
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage.measure import shannon_entropy,label,perimeter,regionprops
from skimage.filters import sobel, threshold_otsu
import pandas as pd
import numpy as np
from PIL import Image, UnidentifiedImageError
import sys, os
sys.path.append('/home/benr/ACT/CW2/py')
from functions import split_data_rf,get_data,galaxy_type
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

# Machine Learning for Galaxy Classification  
This project aims to test how different machine learning methods will perform on a galaxy classifaction task. 

15,000 galaxies from the gz2 dataset will be classified into 4 types of galaxy from the Hubble tuning-fork; E (eliptical), S0 (disk), s(a,b,c) (spirals with no bar) and SB (spirals with bars).

## Qestion 1, Traditional ML method
To test a traditional ML method I will use random forest to try and predict galaxy morphology based on features of the image and pca values.

Radom Forest classifiers can be extremely accurate. Random forest uses the 'votes' of many decision trees to make a prediction, which makes it a good technique for finding patterns through out a dataset. The contribution from many trees means that it is not so easily confused by noise and outliers. 

In [2]:

# get the data from functions.py
df = get_data()
print(df.columns)
#The following ids will be used to identify each galaxy image from SDSS
RA_COL, DEC_COL, ID_COL = 'ra','dec','dr7objid'


Index(['dr7objid', 'ra', 'dec', 'rastring', 'decstring', 'sample', 'gz2_class',
       'total_classifications', 'total_votes',
       't01_smooth_or_features_a01_smooth_count',
       ...
       't11_arms_number_a36_more_than_4_fraction',
       't11_arms_number_a36_more_than_4_weighted_fraction',
       't11_arms_number_a36_more_than_4_debiased',
       't11_arms_number_a36_more_than_4_flag',
       't11_arms_number_a37_cant_tell_count',
       't11_arms_number_a37_cant_tell_weight',
       't11_arms_number_a37_cant_tell_fraction',
       't11_arms_number_a37_cant_tell_weighted_fraction',
       't11_arms_number_a37_cant_tell_debiased',
       't11_arms_number_a37_cant_tell_flag'],
      dtype='object', length=231)


In [3]:
#just incase get rid of any nan cells
df_clean = df.dropna(subset=[RA_COL,DEC_COL,ID_COL])
#cut data to 15,000
df_subset = df_clean.sample(n=15000,random_state=11)
#print the shape
print(df_subset.shape)

(15000, 231)


In [4]:
#get names of columns (labels)
print(df_subset.columns[30:70])


Index(['t02_edgeon_a04_yes_weighted_fraction', 't02_edgeon_a04_yes_debiased',
       't02_edgeon_a04_yes_flag', 't02_edgeon_a05_no_count',
       't02_edgeon_a05_no_weight', 't02_edgeon_a05_no_fraction',
       't02_edgeon_a05_no_weighted_fraction', 't02_edgeon_a05_no_debiased',
       't02_edgeon_a05_no_flag', 't03_bar_a06_bar_count',
       't03_bar_a06_bar_weight', 't03_bar_a06_bar_fraction',
       't03_bar_a06_bar_weighted_fraction', 't03_bar_a06_bar_debiased',
       't03_bar_a06_bar_flag', 't03_bar_a07_no_bar_count',
       't03_bar_a07_no_bar_weight', 't03_bar_a07_no_bar_fraction',
       't03_bar_a07_no_bar_weighted_fraction', 't03_bar_a07_no_bar_debiased',
       't03_bar_a07_no_bar_flag', 't04_spiral_a08_spiral_count',
       't04_spiral_a08_spiral_weight', 't04_spiral_a08_spiral_fraction',
       't04_spiral_a08_spiral_weighted_fraction',
       't04_spiral_a08_spiral_debiased', 't04_spiral_a08_spiral_flag',
       't04_spiral_a09_no_spiral_count', 't04_spiral_a09_no_spiral

The dataset from galaxy zoo provides labels that can be used to idenitify galaxy. 
In the case the following have been selected
* t01_smooth_or_features_a01_smooth_debiased - is the galaxy smooth
* t01_smooth_or_features_a02_features_or_disk_debiased - is the galaxy a disk
* t02_edgeon_a04_yes_debiased - do we see the galaxy edge on 
* t04_spiral_a08_spiral_debiased - does the galaxy have spiral arms
* t03_bar_a06_bar_debiased - does the galaxy have a bar 


In [5]:
#choose best labels to classify galaxies 
labels = ['t01_smooth_or_features_a01_smooth_debiased',
          't01_smooth_or_features_a02_features_or_disk_debiased',
          't02_edgeon_a04_yes_debiased',
          't04_spiral_a08_spiral_debiased',
          't03_bar_a06_bar_debiased'
          ] 


In [6]:
'''create a new column called 'hubble class' and use the above
funtion to get the class for each row''' 

df_subset['hubble_class'] = df_subset.apply(galaxy_type, axis=1)
df_subset = df_subset[df_subset['hubble_class'].notna()]
df_subset['hubble_class'].value_counts()

hubble_class
E            4722
S (a b c)    4210
SB           2477
S(edgeon)    1338
S0            998
Name: count, dtype: int64

In [7]:
# input image data into random forest
# attach lables for each image
x_images = []
y_labels = []
 

Now each galaxy has been placed into a catagory. Next we need to gather the image data for each galaxy in the set and put it into an array.

In [8]:

# folder that stores the images 
img_dir = '/home/benr/ACT/CW2/sdss_images'
# itterate through each galaxy in the set 
for idx,row in df_subset.iterrows():
    obid = row[ID_COL]
    lbl = row['hubble_class']
    #find corrosponding galaxy in the image folder 
    img_path = os.path.join(img_dir, f"{obid}.jpg")
    if not os.path.exists(img_path):
        continue 
    try:
        #use this error incase of currupted images (optional)
        img = Image.open(img_path).convert("L")
    except UnidentifiedImageError:
        print("Skipping corrupted image:", img_path)
        continue
    #store the image and append it to list 
    px_arr = np.array(img,dtype = np.float32)
    x_images.append(px_arr)
    y_labels.append(lbl)


In [9]:
#define your random forest classifier 
rf = RandomForestClassifier(
        n_estimators=900, # number of trees
        max_depth=None, # how many times each tree can split (No limit)
        n_jobs= 1,
        random_state= 11
        
)

In [10]:
# convert to numpy arrays 
x_images = np.array(x_images)
y_labels = np.array(y_labels)
x_images.shape

(13742, 256, 256)

The next step is to create some features for to pass into the random forest model. 

* Eccentricity - measure of how elliptical the galaxy is
                 see https://scikit-image.org/docs/0.25.x/auto_examples/segmentation/plot_regionprops.html?utm_source=chatgpt.com
                 for details.
                 
* mean sobel edge magnitude - The sobel edges is a measure of the gradient
                              at each pixel, we apply this function and then take the mean. This helps distinguish between elepitcal (smooth) galaxies and disks or spirals that will have clearer edges. 

* perimeter - perimeter measures the perimiter of shapes in a binary
              images. This will be smaller in edge on galaxies and larger in the other types. 

* Area - similar logic to perimeter, to help distigues between edge on 
         and other types. 

* asymmetry - the asymmetry is measure by rotating the image by 90 
              degrees and then taking the average of the absolute value of the difference between the two images. 

As well as the above features we also include mean, stadard deviation,median and maximum of the mixal values. 

for details on skimage measures see
https://scikit-image.org/docs/0.25.x/api/skimage.measure.html#

In [11]:
# gather features of images 
x_features = []
for i in x_images:
    #get binary version of image 
    th = threshold_otsu(i)
    bimg = i > th
    # eccentricity 
    lab = label(bimg)
    props = regionprops(lab)
    if len(props) > 0:
        e = props[0].eccentricity  
    else:
        e = 0.0

    edges = sobel(i)
    edg_mean = edges.mean()
     
    p = perimeter(bimg,neighborhood=6)
    A = bimg.sum() 

    rot180 = np.rot90(img, 2)
    asym = np.mean(np.abs(img - rot180))

    feats = np.array([np.mean(i), np.std(i),
                      np.max(i),np.median(i),
                      e,p,A,asym,
                      edg_mean,shannon_entropy(i)] 
                      )
    x_features.append(np.array(feats))
x_features = np.array(x_features)

Now that we have specific features of the images we will use Principle Componant Analysis (PCA) to reduce flattened image vectors to a smaller size whilst keeping the key information about the dataset. 

PCA works by calculating the covariance matrix and finding directions in which the dataset (images) vary the most. All values are then projected onto these new axes reducing the data to a chosen number of dimensions. 

see PCA guide here 
https://www.geeksforgeeks.org/maths/covariance-matrix/

In [12]:

#flatten dataset from [num images, 256,256] to [num images, 256**2] 

xflat = []
for i in range(len(x_images)):
    xflat.append(np.array(x_images[i].flatten()))

In [13]:
# split into training and test sets (see functions.py) 
xtrn_FT,xtst_FT,xtrn_FL,xtst_FL,ytrn,ytst = split_data_rf(x_features,xflat,y_labels,0.1)

In [14]:
#Set up pca 
pca = PCA(n_components=95)
xpca = pca.fit_transform(xtrn_FL)

In [15]:
# join pca features with designed featers 
xtrn = np.hstack([xtrn_FT,xpca]) 

In [16]:
# train the random forest model 
rf.fit(xtrn,ytrn)


,n_estimators,900
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
# join the two feature test sets 
xpca_tst = pca.transform(xtst_FL)
xtst = np.hstack([xtst_FT,xpca_tst])

In [18]:
# make prediction based on test sets 
ypred = rf.predict(xtst)
print(f'accuracy score = {accuracy_score(ytst,ypred)}') 

accuracy score = 0.5207272727272727


As the results show, the random forest model only made correct predictions for half of the galaxies in the dataset. This is because the 256x256 pixel images mean that the model has a huge amount of input features; even after PCA. Each tree is trying to make decisions on all of these features which makes it difficult to find more general patterns in the image set. 

Another issue is that random forest does not have the ability to properly assess spacial features, it is not ablle to find links between certain edges and brightnesses in the same way a more sophisticated convolutional neural network might be able to. 